<a href="https://colab.research.google.com/github/oktaviacitra/classification/blob/main/Iris.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import load_iris

iris = load_iris()
X, y = iris.data, iris.target

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier

pipeline = Pipeline(
    [
        ('scaler', StandardScaler()), 
        ('model', DecisionTreeClassifier(random_state=21))
    ]
)

In [3]:
pipeline.get_params()

{'memory': None,
 'steps': [('scaler', StandardScaler()),
  ('model', DecisionTreeClassifier(random_state=21))],
 'verbose': False,
 'scaler': StandardScaler(),
 'model': DecisionTreeClassifier(random_state=21),
 'scaler__copy': True,
 'scaler__with_mean': True,
 'scaler__with_std': True,
 'model__ccp_alpha': 0.0,
 'model__class_weight': None,
 'model__criterion': 'gini',
 'model__max_depth': None,
 'model__max_features': None,
 'model__max_leaf_nodes': None,
 'model__min_impurity_decrease': 0.0,
 'model__min_samples_leaf': 1,
 'model__min_samples_split': 2,
 'model__min_weight_fraction_leaf': 0.0,
 'model__random_state': 21,
 'model__splitter': 'best'}

In [4]:
from sklearn.model_selection import GridSearchCV

param_grid = {'model__splitter': ['best', 'random'],
              'model__criterion': ['gini', 'entropy', 'log_loss'],
              'model__min_samples_leaf': [1, 1.5, 2, 2.5, 3],
              'model__min_samples_leaf': [0.5, 1, 1.5],
              'model__min_weight_fraction_leaf': [0.0, 0.01, 0.001],
              'model__min_impurity_decrease': [0.0, 0.01],
              'model__ccp_alpha': [0.0099, 0.01, 0.0]
              }

search = GridSearchCV(
    estimator = pipeline,
    param_grid = param_grid,
    n_jobs=-1,
    scoring="neg_mean_squared_error",
    cv=5,
    verbose=3
)

search.fit(X, y)

Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:372: FitFailedWarning: 
900 fits failed out of a total of 1620.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
540 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/pipeline.py", line 394, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/tree/_classes.py", line 942, in fit
    X_idx_sorted=X_idx_sorted,
  File "/usr/local/lib/python3.7/dist-package

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model',
                                        DecisionTreeClassifier(random_state=21))]),
             n_jobs=-1,
             param_grid={'model__ccp_alpha': [0.0099, 0.01, 0.0],
                         'model__criterion': ['gini', 'entropy', 'log_loss'],
                         'model__min_impurity_decrease': [0.0, 0.01],
                         'model__min_samples_leaf': [0.5, 1, 1.5],
                         'model__min_weight_fraction_leaf': [0.0, 0.01, 0.001],
                         'model__splitter': ['best', 'random']},
             scoring='neg_mean_squared_error', verbose=3)

In [6]:
search.best_params_, search.best_estimator_

({'model__ccp_alpha': 0.0099,
  'model__criterion': 'gini',
  'model__min_impurity_decrease': 0.0,
  'model__min_samples_leaf': 1,
  'model__min_weight_fraction_leaf': 0.0,
  'model__splitter': 'best'},
 Pipeline(steps=[('scaler', StandardScaler()),
                 ('model',
                  DecisionTreeClassifier(ccp_alpha=0.0099, random_state=21))]))